## Validation plot answer sheet

Compute a plot of the position-dependent PSF size (we'll define this as the trace we computed in the last notebook)
* Make the band an input argument, and run plots for the g-band and the i-band

Hints 
* scipy.binned_statistic_2d is the function I like to use for this
* You can use the code for the data read you used last challenge
* If you were reading large amounts of data and don't want to repeat the read, just pass the data in through your input dictionary



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import time
sys.path.insert(0,'/global/cfs/projectdirs/lsst/groups/SRV/gcr-catalogs-test')
import GCRCatalogs
sys.path.insert(0,'/global/cfs/projectdirs/lsst/groups/SRV/descqa')
import descqa
import glob
from PIL import Image
from IPython.display import display


%matplotlib inline

In [ ]:
from descqa.base import BaseValidationTest, TestResult
import numpy as np
import healpy as hp
import os
import matplotlib.pyplot as plt
import matplotlib as mpl


from scipy.stats import binned_statistic_2d as bs2d
def trace(ixx,ixy,iyy):
    return (ixx+iyy)

    
class Tutorial3(BaseValidationTest):
    """
    validation test to plot RA/DEC
    """
    def __init__(self, **kwargs): #pylint: disable=W0231
        # get the inputs
        self.ra = kwargs.get('ra')
        self.dec = kwargs.get('dec')
        self.band = kwargs.get('band')
        self.catalog_filters = kwargs.get('catalog_filters', [])
        self.quantities =  kwargs.get('quantities', [])



    def run_on_single_catalog(self, catalog_instance, catalog_name, output_dir):


        if not catalog_instance.has_quantities([self.ra, self.dec]):
            return TestResult(skipped=True)
        filters=[]
        for i, filt in enumerate(self.catalog_filters):
            filters.append(filt)

            
        # read in the data 
        data = catalog_instance.get_quantities(self.quantities,filters=filters, return_iterator=False)
        
        data['T_PSF'] = trace(data[self.band+'_ixxPSF'],data[self.band+'_ixyPSF'],data[self.band+'_iyyPSF'])
        
        plt.title(self.band+'-band PSF size variation')
        val, xedge, yedge, bincount = bs2d(data["coord_ra"],data["coord_dec"],data['T_PSF'],statistic='mean',bins=100)
        plt.imshow(val,extent=[xedge[0],xedge[-1],yedge[0],yedge[-1]])
        plt.colorbar()
        plt.xlabel('RA (degrees)')
        plt.ylabel('dec (degrees)')
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, 'test_histogram.png'))
        plt.close()


        return TestResult(inspect_only=True, summary='made a histogram to inspect!')

In [ ]:
band = 'g'
dict_config = {}
dict_config['ra'] = 'coord_ra'
dict_config['dec'] = 'coord_dec'
dict_config['band'] = band
dict_config['quantities'] = ["coord_ra","coord_dec", band+"_ixxPSF", band+"_ixyPSF", band+"_iyyPSF",band+"_cModelFlux"]
dict_config['catalog_filters'] =  ['detect_isPrimary','refExtendedness==1']


catalog = 'lsst_object'
catalog_instance =  GCRCatalogs.load_catalog(catalog)   
output_dir_this = './test_direc'  


validation3 = Tutorial3(**dict_config)
test_result = validation3.run_on_single_catalog(catalog_instance, catalog, output_dir_this)

for image in glob.glob( output_dir_this + '/*.png'):
    print(image)
    display(Image.open(image))


# Challenges!

Okay you should have everything you need to get started looking at and validating the data. Go ahead and make your own plots! 

I have some suggestions below that the SRV group would love to get some new validation tests drafted for, but anything you or your working group wants to know about the data before it goes through an analysis pipeline is fair game! Or just test out using the DP0.2 data for your own analysis 

Some FAQs:
* There are no photometric redshifts available for this data at this part of the pipleline, see the RAIL tutorial for more details on those
* There are no metadetect shears available due to development timelines, however HSM moments are available in the catalog
* If you want direct comparison tests we can match to the truth catalog for DP0.2 (and with SSI catalogs for testing), ask me to set up a quick yaml file if you'd like to draft one for DP0.2 vs the cosmoDC2 inputs (note to self let's make this!) 

## Tasks! 

* Estimate the 5-sigma limiting magnitude (using the psfFlux and definitions as described in https://dmtn-296.lsst.io/) and make a plot of the average value over RA/dec
    * Repeat for all bands (grizuy) 
* Inspect the cModel magnitudes, calib magnitudes and gaap magnitudes and colors against the truth values
    * What conclusions can you draw about the best colors to use?
    * Do these agree with the suggestions noted in the photometry flags section of https://lsstdesc.org/srv-dp02/tutorials/Rubin/tutorial7_SRV_releases.html ?
* Find a few of the brightest stars, using extendedness and magnitude cuts. Look for the RA/dec position of these and plot the galaxy density in this location. What do you see?
    * Extra credit exercise for those with DP0 delegate accounts, use the tutorials to find the tract/patch of this co-ordinate and locate and plot the deepcoadd image for this area
* Compare the gaap photometry for different apertures including the optimal aperture
    * Look through here to understand the method: https://arxiv.org/pdf/1507.00738
    * Run the srv_gaap validation code on the dp02_object_single_tract catalog (https://github.com/LSSTDESC/descqa/blob/SRV/descqa/srv_gaap.py) and see if our results agree
    * Have a look [here](https://lsstdesc.org/srv-dp02/tutorials/Rubin/tutorial7_SRV_releases.html) and the linked documentation. Do you see hints of this issue when looking at the data? How can we expand our testing to find this?
